In [23]:
import gurobipy as grb
import numpy as np

#### Notes
- costs could be on a per-mile basis, and then use projected miles to determine an actual annualized cost amount
- dummy costs are just random numbers for now. Will need to replace with functions that can project out throguh the planning period

# Data

In [81]:
#random set of vehicles. v_p (v prime) is the set of ICE vehicles. v_pp (v double prime) is the set of EV alternatives
V_P = ['FORD FOCUS','DODGE JOURNEY','HONDA CR-V']
V_PP = ['FORD FUSION HYBRID','CHRYSLER PACIFICA HYBRID','HONDA CR-V HYBRID']
V = V_P+V_PP

print(f'ICE only: {V_P}')
print(f'EVs: {V_PP}')
print(f'All: {V}')

ICE only: ['FORD FOCUS', 'DODGE JOURNEY', 'HONDA CR-V']
EVs: ['FORD FUSION HYBRID', 'CHRYSLER PACIFICA HYBRID', 'HONDA CR-V HYBRID']
All: ['FORD FOCUS', 'DODGE JOURNEY', 'HONDA CR-V', 'FORD FUSION HYBRID', 'CHRYSLER PACIFICA HYBRID', 'HONDA CR-V HYBRID']


In [83]:
#random set of departments
D = ['Recreation and Parks (Recreation Services)',
     'Parking Enforcement Unit',
     'Department of Aging (Facilities)']
D

['Recreation and Parks (Recreation Services)',
 'Parking Enforcement Unit',
 'Department of Aging (Facilities)']

In [18]:
#years in the planning horizon
T = [t for t in range(2021,2038)]

In [85]:
#generate random inventory
I = {}
for d in D:
    for v_p in V_P:
        I[d,v_p] = np.random.randint(20,70)
    for v_pp in V_PP:
        I[d,v_pp] = 0
I

{('Recreation and Parks (Recreation Services)', 'FORD FOCUS'): 22,
 ('Recreation and Parks (Recreation Services)', 'DODGE JOURNEY'): 38,
 ('Recreation and Parks (Recreation Services)', 'HONDA CR-V'): 20,
 ('Recreation and Parks (Recreation Services)', 'FORD FUSION HYBRID'): 0,
 ('Recreation and Parks (Recreation Services)', 'CHRYSLER PACIFICA HYBRID'): 0,
 ('Recreation and Parks (Recreation Services)', 'HONDA CR-V HYBRID'): 0,
 ('Parking Enforcement Unit', 'FORD FOCUS'): 26,
 ('Parking Enforcement Unit', 'DODGE JOURNEY'): 53,
 ('Parking Enforcement Unit', 'HONDA CR-V'): 51,
 ('Parking Enforcement Unit', 'FORD FUSION HYBRID'): 0,
 ('Parking Enforcement Unit', 'CHRYSLER PACIFICA HYBRID'): 0,
 ('Parking Enforcement Unit', 'HONDA CR-V HYBRID'): 0,
 ('Department of Aging (Facilities)', 'FORD FOCUS'): 53,
 ('Department of Aging (Facilities)', 'DODGE JOURNEY'): 25,
 ('Department of Aging (Facilities)', 'HONDA CR-V'): 58,
 ('Department of Aging (Facilities)', 'FORD FUSION HYBRID'): 0,
 ('Depar

In [87]:
#cost factors for consumables (C), maintenance (M), procurement (P), emissions (E)
dimensions,C,M,P,E = grb.multidict({(v,d,t): [np.random.randint(1,4), #random consumables cost per mile 
                             np.random.randint(200,400), #random maintenance cost per vehicle
                             np.random.randint(10000,40000), #random procurement cost per vehicle
                             np.random.randint(1,20),] #random emissions 
                            for v in V for d in D for t in T
                  })
#example consumables. ---I think this could actually just be a projected cost per mile for each vehicle type (based on what type of consumable they use). Then rolled up
C

{('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2021): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2022): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2023): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2024): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2025): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2026): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2027): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2028): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2029): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2030): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2031): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2032): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2033): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 20

In [88]:
#annual  procurement and maintenance cost per charging station
dimensions,P_s,M_s = grb.multidict({(t): [
                             np.random.randint(20000,40000), #random maintenance cost per vehicle
                             np.random.randint(5000,10000), #random procurement cost per vehicle
                             ] for t in T
                  })

P_s

{2021: 25860,
 2022: 38678,
 2023: 22194,
 2024: 26583,
 2025: 25425,
 2026: 33832,
 2027: 36602,
 2028: 36719,
 2029: 37522,
 2030: 20191,
 2031: 38878,
 2032: 23569,
 2033: 32938,
 2034: 24884,
 2035: 21629,
 2036: 34158,
 2037: 35056}

In [89]:
#! model will probs be infeasible if this number is greater than inventory

#number of vehicles of type v_p that must remain on hand in dpt d in year t due to lack of miles eligibility for replacement
dimensions,N = grb.multidict({(v_p,d,t): [
                             np.random.randint(0,4), 
                             ] for v_p in V_P for d in D for t in T
                  })

N

{('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2021): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2022): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2023): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2024): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2025): 0,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2026): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2027): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2028): 1,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2029): 3,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2030): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2031): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2032): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 2033): 2,
 ('FORD FOCUS', 'Recreation and Parks (Recreation Services)', 20

In [61]:
#weight to apply to each objective
w = [0.5,0.5]

In [64]:
#target emissions number in final year (based on % of baseline)
Q = 3

In [90]:
#annual budget amount
B = {t:np.random.randint(200000,400000) for t in T}
B

{2021: 304115,
 2022: 340471,
 2023: 285302,
 2024: 222166,
 2025: 338292,
 2026: 342157,
 2027: 376259,
 2028: 268417,
 2029: 361534,
 2030: 203906,
 2031: 261272,
 2032: 242582,
 2033: 331057,
 2034: 327659,
 2035: 230808,
 2036: 225375,
 2037: 387886}

In [67]:
#maximum number of veihcles that a single charging station can service
G = 1000

In [91]:
#whether or not v_pp is a suitable replacement for v_p
R = {('FORD FOCUS', 'FORD FUSION HYBRID'): 1,
     ('FORD FOCUS', 'CHRYSLER PACIFICA HYBRID'): 0,
     ('FORD FOCUS', 'HONDA CR-V HYBRID'): 0,
     ('DODGE JOURNEY', 'FORD FUSION HYBRID'): 0,
     ('DODGE JOURNEY', 'CHRYSLER PACIFICA HYBRID'): 1,
     ('DODGE JOURNEY', 'HONDA CR-V HYBRID'): 0,
     ('HONDA CR-V', 'FORD FUSION HYBRID'): 0,
     ('HONDA CR-V', 'CHRYSLER PACIFICA HYBRID'): 0,
     ('HONDA CR-V', 'HONDA CR-V HYBRID'): 1}

# Model 

In [2]:
m = grb.Model('CARNET')

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only


## Decision Variables

In [97]:
#number of vehicles of type v to HAVE ON HAND in dpt d in year t (both ICE and EV)
x = m.addVars(V,D,T,vtype=grb.GRB.INTEGER,name='x')

#number of vehicles of type v to PROCURE in dpt d in year t (both ICE and EV)
y = m.addVars(V,D,T,vtype=grb.GRB.INTEGER,name='y')

#number of charging stations to have in operation in year t
z = m.addVars(T,vtype=grb.GRB.INTEGER,name='z')

#number of charging stations to build in year t
s = m.addVars(T,vtype=grb.GRB.INTEGER,name='s')

#amount over budget in year t
P_B = m.addVars(T,vtype=grb.GRB.CONTINUOUS,name='P_B')

#! may only need for final year
#amount over emissions target in year t
P_B = m.addVars(T,vtype=grb.GRB.CONTINUOUS,name='P_B')

## Objective

## Constraints 

## Solution & Output Analysis 